In [1]:
from pyrosm import OSM
from pyrosm import get_data
import openeo
import matplotlib.pyplot as plt

import os

In [2]:
city = "Bonn"

USE_LOCAL = False

In [3]:

if not os.path.exists(os.path.join('data', city)):
    USE_LOCAL = False
if not USE_LOCAL:
    os.makedirs(os.path.join('data', city), exist_ok=True)

In [4]:
# Download data for given city, or load if already downloaded
fp = get_data(city, directory=os.path.join('data', city))
osm = OSM(fp)
print("Loaded OSM data for " + city)

# Filter buildings
buildings = osm.get_buildings()
print("Filtered buildings in " + city)

# Get boundaries of the city
min_x, min_y, max_x, max_y = buildings.total_bounds
print("Coordinates of " + city + ": " + str(min_x),
      str(min_y), str(max_x), str(max_y))

Loaded OSM data for Bonn
Filtered buildings in Bonn
Coordinates of Bonn: 6.800000190734863 50.470001220703125 7.359999179840088 50.900001525878906


In [7]:
if not USE_LOCAL:

    # create connection
    connection = openeo.connect("https://openeo.cloud")

    # authenticate
    connection.authenticate_oidc()

    # define time
    temporal_extent = ["2023-03-01", "2024-10-30"]

    # define coordinates
    bbox = {"west": min_x, "south": min_y, "east": max_x, "north": max_y}

    # define the collection ID
    collection_id = "SENTINEL2_L2A"

    # define bands
    bands = ["B04", "B03", "B02", ]

    # load the data collection
    datacube = connection.load_collection(
        collection_id=collection_id,
        spatial_extent=bbox,
        temporal_extent=temporal_extent,
        bands=bands,
        max_cloud_cover=1
    )

    # TODO deal with no data found in timeinterval because of small cloud cover threshold of 1.

    # download and safe data
    datacube.download(os.path.join("data", city, "sat_images.nc"))

Authenticated using refresh token.


ChunkedEncodingError: ('Connection broken: IncompleteRead(163833 bytes read, 30347 more expected)', IncompleteRead(163833 bytes read, 30347 more expected))